In [ ]:
from io import StringIO
import json

import boto3
import pandas as pd

def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """" Perform transformation with the taxi data.
    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame holding the daily taxi trips

    Returns
    -------
    pd.DataFrame
        The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame")

    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract",
                     "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                                "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")

    return taxi_trips
    
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master, company_master: pd.DataFrame) -> pd.DataFrame:
    """Update the taxi_trips DataFrame with the company_master and the payment_type_master ids and delete the string columns

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame with the daily taxi trips.
    payment_type_master : pd.DataFrame
        The payment type master table.
    company_master : pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        The taxi trips data with only payment_type_id and company_ide, without company or payment_type values
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id
    
    
def updated_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """Extend master DataFrame with new values if there are any.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily taxi trips
    master : pd.DataFrame
        Dataframe holding the payment_type_master data.
    id_column : str
        The id column of the master DataFrame
    value_column : str
         The name of the column in master_df containing the values.   

    Returns
    -------
    pd.DataFrame
        The updated master data, if new values are in the taxi data, they will be loaded to it.
    """

    max_id = master[id_column].max()

    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
        value_column: new_values_list
    })

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master    

def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """Make transformations on the daily weather api response

    Parameters
    ----------
    weather_data : json
        The daily weather data from the Open Meteo API.

    Returns
    -------  
    pd.DataFrame
        A DataFrame representation of the data
    """

    weather_data_filtered = {
    "datetime": weather_data["hourly"]["time"],
    "temperature": weather_data["hourly"]["temperature_2m"],
    "wind_speed": weather_data["hourly"]["wind_speed_10m"],
    "rain": weather_data["hourly"]["rain"],
    "precipitation": weather_data["hourly"]["precipitation"],
    }

    weather_df = pd.DataFrame(weather_data_filtered)

    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    return weather_df 
    
def read_csv_froms3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """Downloads a csv file from an S3 bucket.
    
    Parameters
    ----------
    bucket  : str
        The bucket where the files at.
    path : str
        The folders to the file.
    filename : str
        The name of the file
    
    Returns
    -------
    pd.DataFrame
        A Dataframe of the downloaded file.
    """
    s3 = boto3.client("s3")
    full_path = f"{path}{filename}"
    
    
    object = s3.get_object(Bucket=bucket, Key=full_path)
    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))
    
    return output_df
   
def upload_dataframe_to_s3(dataframe: pd.DataFrame, bucket: str, path: str):
    """
    Uploads a dataframe to the specified S3 path.
    
    Parameters
    
    dataframe : pd.DataFrame
        The dataframe to be uploaded.
        
    bucket : str
        Name of the S3 bucket where we want to store the files  
    
    path: str
        Path within the bucket to upload the files
    Returns    
    --------
    None
    """
   
    s3 = boto3.client("s3")
    
    buffer = StringIO()
    dataframe.to_csv(buffer, index=False)
    df_content = buffer.getvalue()
    s3.put_object(Bucket=bucket, Key=path, Body=df_content)
    
def upload_master_data_to_s3(bucket: str, path: str, file_type: str, dataframe: pd.DataFrame):
    """
    Uploads master data (payment_type or company) to S3. Copies the previous version and creates the new one.
    
    Parameters
    ----------
    bucket : str
        Name of the S3 bucket where we want to store the files
        
    path : str
        Path within the bucket to upload the files
        
    file_type : str
        Either "company" or "payment_type".
        
    dataframe: pd.DataFrame
        The dataframe to be uploaded
        
    Returns
    -------
    None
    """
    s3 = boto3.client("s3")
    
    master_file_path = f"{path}{file_type}_master.csv"
    previous_master_file_path = f"transformed_data/master_data_previous_version/{file_type}_master_previous_version.csv"
    
    s3.copy_object(
        Bucket=bucket,
        CopySource={"Bucket": bucket, "Key": master_file_path},
        Key=previous_master_file_path
    )
    
    upload_dataframe_to_s3(bucket=bucket, dataframe=dataframe, path=master_file_path)
    
def upload_and_move_file_on_s3(
    dataframe: pd.DataFrame,
    datetime_col: str,
    bucket: str,
    file_type: str,
    filename: str,
    source_path: str,
    target_path_raw: str,
    target_path_transformed: str,
    ):
    
    """
    Uploads a DataFrame to S3 and then moves a file from the base folder to another
    
    Parameters
    ----------
    bucket : str
        Name of the S3 bucket.
    
    file_type : str
        "weather" or "trips"
    
    source_path : str
        Source path within the bucket.
    
    target_path_transformed : str
        Target path within the bucket where the raw data would go.
    
    target_path_raw : str
        Target path within the bucket where the raw data would go.
        
    filename : str
        Name of the file to be uploaded or moved
    
    dataframe : pd.DataFrame, optional
        The DataFrame to be uploaded
        
    datetime_col : str, optional
        Datetime column name, which we derive the date for the filename
    
    Returns
    -------
    None
    """
    s3 = boto3.client("s3")
    
    formatted_date = dataframe[datetime_col].iloc[0].strftime("%Y-%m-%d")
    new_path_with_filename = f"{target_path_transformed}{file_type}_{formatted_date}.csv"
      
      # transformed_data/taxi_trips/taxi_trips/ >>> target path
      # taxi_trips >>>> file_type
      # 2023-09-06 >>> formatted_date
      
    upload_dataframe_to_s3(bucket=bucket, dataframe=dataframe, path=new_path_with_filename)
    
    
    s3.copy_object(
        Bucket=bucket,
        CopySource={"Bucket": bucket, "Key": f"{source_path}{filename}"},
        Key=f"{target_path_raw}{filename}"
    )
    
    s3.delete_object(Bucket=bucket, Key=f"{source_path}{filename}")
 
        
def lambda_handler(event, context):
    s3 = boto3.client("s3")
    
    bucket = "cubix-chicago-taxi-szr"
    raw_weather_folder = "raw_data/to_processed/weather_data/"
    raw_taxi_trips_folder = "raw_data/to_processed/taxi_data/"
    target_taxi_trips_folder = "raw_data/processed/taxi_data/"
    target_weather_folder = "raw_data/processed/weather_data/"
    
    transformed_taxi_trips_folder = "transformed_data/taxi_trips/"
    transform_weather_folder = "transformed_data/weather/"
    
    payment_type_master_folder = "transformed_data/payment_type/"
    company_master_folder = "transformed_data/company/"
    
    payment_type_master_file_name = "payment_type_master.csv"
    company_master_file_name = "company_master.csv"
    
    
    payment_type_master = read_csv_froms3(bucket=bucket, path=payment_type_master_folder, filename=payment_type_master_file_name)
    company_master = read_csv_froms3(bucket=bucket, path=company_master_folder, filename=company_master_file_name)
    
    
    # Taxi data transformation and loading
    
    for file in s3.list_objects(Bucket=bucket, Prefix=raw_taxi_trips_folder)["Contents"]:
        taxi_trip_key = file["Key"]
        
        if taxi_trip_key.split("/")[-1] != "":
            if taxi_trip_key.split(".")[1] == "json":
            
                filename = taxi_trip_key.split("/")[-1]
                
                response = s3.get_object(Bucket=bucket, Key=taxi_trip_key)
                content = response["Body"]
                taxi_trip_data_json = json.loads(content.read())
                
                taxi_trip_data_raw = pd.DataFrame(taxi_trip_data_json)
                taxi_trips_transformed = taxi_trips_transformations(taxi_trip_data_raw) 
                
                company_master_update = updated_master(taxi_trips_transformed, company_master, "company_id", "company")
                payment_type_master_updated = updated_master(taxi_trips_transformed, payment_type_master, "payment_type_id", "payment_type")
                
                taxi_trips = update_taxi_trips_with_master_data(taxi_trips_transformed, payment_type_master_updated, company_master_update)
                
                upload_and_move_file_on_s3(
                        dataframe=taxi_trips,
                        datetime_col="datetime_for_weather",
                        bucket=bucket,
                        file_type="taxi",
                        filename=filename,
                        source_path=raw_taxi_trips_folder,
                        target_path_raw=target_taxi_trips_folder,
                        target_path_transformed=transformed_taxi_trips_folder
                    )
                print("Tax_trips is uploaded and moved")
                    
                upload_master_data_to_s3(bucket=bucket, path=payment_type_master_folder, file_type="payment_type", dataframe=payment_type_master_updated)
                print("payment_type_master has been updated")
                upload_master_data_to_s3(bucket=bucket, path=company_master_folder, file_type="company", dataframe=company_master_update)
                print("company_master has been updated")
    
    # Weather data transformation and loading
    
    for file in s3.list_objects(Bucket=bucket, Prefix=raw_weather_folder)["Contents"]:
        weather_key = file["Key"]
        
        if weather_key.split("/")[-1] != "":
            if weather_key.split(".")[1] == "json":
                
                filename = weather_key.split("/")[-1]
                
                response = s3.get_object(Bucket=bucket, Key=weather_key)
                content = response["Body"]
                weather_data_json = json.loads(content.read())
                    
                weather_data = transform_weather_data(weather_data_json)
                    
                    
                upload_and_move_file_on_s3(
                        dataframe=weather_data,
                        datetime_col="datetime",
                        bucket=bucket,
                        file_type="weather",
                        filename=filename,
                        source_path=raw_weather_folder,
                        target_path_raw=target_weather_folder,
                        target_path_transformed=transform_weather_folder
                    )
                print("Weather is uploaded and moved")
        